# Part 1 - Scraping neighborhood data from Wikipedia

In [14]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import folium

In [15]:
# Scraping the wikipedia website
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
soup = BeautifulSoup(response.content)

# Converting to human readable content
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df = df[0].to_json()
df[0:1000]

'{"Postcode":{"0":"M1A","1":"M2A","2":"M3A","3":"M4A","4":"M5A","5":"M6A","6":"M6A","7":"M7A","8":"M8A","9":"M9A","10":"M1B","11":"M1B","12":"M2B","13":"M3B","14":"M4B","15":"M4B","16":"M5B","17":"M5B","18":"M6B","19":"M7B","20":"M8B","21":"M9B","22":"M9B","23":"M9B","24":"M9B","25":"M9B","26":"M1C","27":"M1C","28":"M1C","29":"M2C","30":"M3C","31":"M3C","32":"M4C","33":"M5C","34":"M6C","35":"M7C","36":"M8C","37":"M9C","38":"M9C","39":"M9C","40":"M9C","41":"M1E","42":"M1E","43":"M1E","44":"M2E","45":"M3E","46":"M4E","47":"M5E","48":"M6E","49":"M7E","50":"M8E","51":"M9E","52":"M1G","53":"M2G","54":"M3G","55":"M4G","56":"M5G","57":"M6G","58":"M7G","59":"M8G","60":"M9G","61":"M1H","62":"M2H","63":"M3H","64":"M3H","65":"M3H","66":"M4H","67":"M5H","68":"M5H","69":"M5H","70":"M6H","71":"M6H","72":"M7H","73":"M8H","74":"M9H","75":"M1J","76":"M2J","77":"M2J","78":"M2J","79":"M3J","80":"M3J","81":"M4J","82":"M5J","83":"M5J","84":"M5J","85":"M6J","86":"M6J","87":"M7J","88":"M8J","89":"M9J","90":"

## Converting and cleaning the dataframe

In [16]:
# Converting to DataFrame
data = json.loads(df)
data = pd.DataFrame(data)
data.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [17]:
# Dropping the empty Borough rows
data = data[data.Borough != 'Not assigned'].reset_index(drop=True)
data.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [18]:
# Checking for neighborhoods which are 'Not Assigned'
data[data.Neighborhood == 'Not assigned']

,Postcode,Borough,Neighborhood
5,M7A,Queen's Park,Not assigned


In [19]:
# Assigning missing neighborhood name to it's borough
data.loc[[5], ['Neighborhood']] = "Queen's Park"

In [20]:
# Merging the rows by postcode name
data = data.groupby(['Postcode', 'Borough'], as_index=False)['Neighborhood'].apply(', '.join).reset_index()

In [21]:
# Renaming the column
data = data.rename(columns={0: 'Neighborhood'})
data.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
data.shape

(103, 3)

# Part 2 - getting the latitude and longitude

In [23]:
# Read the coordinate data
loc_data = pd.read_csv('Geospatial_Coordinates.csv')
loc_data = loc_data.rename(columns={'Postal Code': 'Postcode'})
loc_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
# Merging the dataframes by Postcode
df = pd.merge(data, loc_data, how='outer', on='Postcode')
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [25]:
df.shape

(103, 5)

# Part 3 - exploring and clustering neighborhoods in Toronto

In [26]:
# Get the coordinates of Toronto

from geopy.geocoders import Nominatim

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="neigh_toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [27]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [28]:
# Get Foursquare credentials

CLIENT_ID = 'JLPFMODERBG0EMQHWGJ5KIZPLE3T2FCATEDLKSQZHOUM0WUU' # your Foursquare ID
CLIENT_SECRET = '1PALU5LZP0FNWBFJUXTHHDTHDGLALBZRVE0Y1CLQ20C0REJF' # your Foursquare Secret
VERSION = '20191231' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JLPFMODERBG0EMQHWGJ5KIZPLE3T2FCATEDLKSQZHOUM0WUU
CLIENT_SECRET:1PALU5LZP0FNWBFJUXTHHDTHDGLALBZRVE0Y1CLQ20C0REJF


### Defining a function that returns nearby venues of a given locations

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    """Function that returns nearby venues of a given locations"""
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
# get the venues for given coordinates and name

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [31]:
# size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2233, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [32]:
# how many venues were returned in a neighborhood
toronto_venues.groupby('Neighborhood')['Venue'].count().reset_index()

,Neighborhood,Venue
0,"Adelaide, King, Richmond",100
1,Agincourt,5
2,"Agincourt North, L'Amoreaux East, Milliken, St...",2
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",10
4,"Alderwood, Long Branch",10
...,...,...
94,Willowdale West,6
95,Woburn,4
96,"Woodbine Gardens, Parkview Hill",11
97,Woodbine Heights,9


In [33]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# get the index of neighborhood column
cols = toronto_onehot.columns.to_list()
print(cols.index('Neighborhood'))

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[191]] + list(toronto_onehot.columns[0:191]) + list(toronto_onehot.columns[192:275])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

194
(2233, 275)


,Moving Target,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Count the mean value of every venue
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Moving Target,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0


### Check if number of Neighborhoods is the same

In [36]:
toronto_grouped.shape

(99, 275)

In [37]:
len((toronto_venues['Neighborhood'].unique()))

99

### Defining a support function and assigning top venues

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Restaurant,Sushi Restaurant,Burger Joint,Asian Restaurant,Cosmetics Shop
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Discount Store,Sandwich Place,Beer Store,Japanese Restaurant,Pharmacy,Fast Food Restaurant,Dumpling Restaurant
4,"Alderwood, Long Branch",Pizza Place,Pub,Pharmacy,Sandwich Place,Dance Studio,Athletics & Sports,Skating Rink,Coffee Shop,Gym,Comic Shop


## Clustering Neighborhoods

In [61]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0, n_init=10).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 1, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [62]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.shape)

toronto_merged.head() # check the last columns!

(103, 16)


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Moving Target,Bar,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Rental Car Location,Intersection,Electronics Store,Mexican Restaurant,Breakfast Spot,Pizza Place,Medical Center,Dim Sum Restaurant,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Indian Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Gas Station,Bakery,Fried Chicken Joint,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Bank,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant


In [63]:
# drop aby NaNs which couldnt be downloaded
toronto_merged = toronto_merged.dropna()
toronto_merged.shape

(100, 16)

In [64]:
from geopy.geocoders import Nominatimimport matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

SyntaxError: invalid syntax (<ipython-input-64-f118aeebe535>, line 1)

## Examining the clusters

In [65]:
# Merging all clusters into one dataframe

toronto_clustered = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


for n in range(1, 7):
    
    toronto_temp = toronto_merged.loc[toronto_merged['Cluster Labels'] == n, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
    toronto_clustered = toronto_clustered.append(toronto_temp, ignore_index=True)
    
toronto_clustered.reset_index(drop=True)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Silver Hills, York Mills",0.0,Cafeteria,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Falafel Restaurant
1,"Rouge, Malvern",1.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant
2,"Clairlea, Golden Mile, Oakridge",1.0,Bakery,Park,Metro Station,Bus Station,Bus Line,Fast Food Restaurant,Soccer Field,Gay Bar,Gastropub,Eastern European Restaurant
3,"Agincourt North, L'Amoreaux East, Milliken, St...",1.0,Park,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
4,York Mills West,1.0,Park,Bar,Bank,Convenience Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dessert Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Westmount,2.0,Pizza Place,Middle Eastern Restaurant,Coffee Shop,Sandwich Place,Discount Store,Chinese Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant
96,"Albion Gardens, Beaumond Heights, Humbergate, ...",2.0,Grocery Store,Pizza Place,Fried Chicken Joint,Discount Store,Sandwich Place,Beer Store,Japanese Restaurant,Pharmacy,Fast Food Restaurant,Dumpling Restaurant
97,Northwest,2.0,Rental Car Location,Bar,Drugstore,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
98,"Humber Bay, King's Mill Park, Kingsway Park So...",3.0,Baseball Field,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop


In [66]:
# counting the number of venues in a cluster
toronto_clustered.groupby('Cluster Labels')['Neighborhood'].count().reset_index()

,Cluster Labels,Neighborhood
0,0.0,1
1,1.0,17
2,2.0,80
3,3.0,2
